# Seznam - Predicting the transaction volume

*NOTE: Due to the size of the dataset, this notebook will not run on MyBinder.*

Seznam is a Czech company with a scope similar to Google. The purpose of this notebook is to analyze data from Seznam's wallet, predicting the transaction volume.

Summary:

- Prediction type: __Regression model__
- Domain: __E-commerce__
- Prediction target: __Transaction volume__ 
- Population size: __1,458,233__

_Author: Dr. Patrick Urbanke_

# Background

Seznam is a Czech company with a scope similar to Google. The purpose of this notebook is to analyze data from Seznam's wallet, predicting the transaction volume.

Since the dataset is in Czech, we will quickly translate the meaning of the main tables:

- *dobito*: contains data on prepayments into a wallet
- *probehnuto*: contains data on charges from a wallet
- *probehnuto_mimo_penezenku*: contains data on charges, from sources other than a wallet

The dataset has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/Seznam) (Motl and Schulte, 2015).

We will benchmark [getML](https://www.getml.com) 's feature learning algorithms against [featuretools](https://www.featuretools.com), an open-source implementation of the propositionalization algorithm, similar to getML's FastProp.

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set up your session:

In [1]:
import copy
import os
from urllib import request

import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  

import featuretools
import getml

getml.set_project('seznam')




Connected to project 'seznam'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data:

In [4]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="Seznam",
    port=3306,
    user="guest",
    password="relational"
)

conn

OSError: MySQL error (2005) [HY000] Unknown MySQL server host 'relational.cvut.cz' (-2)

In [ ]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [ ]:
dobito = load_if_needed("dobito")
probehnuto = load_if_needed("probehnuto")
probehnuto_mimo_penezenku = load_if_needed("probehnuto_mimo_penezenku")

In [ ]:
dobito

In [ ]:
probehnuto

In [ ]:
probehnuto_mimo_penezenku

### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [ ]:
dobito.set_role("client_id", getml.data.roles.join_key)
dobito.set_role("month_year_datum_transakce", getml.data.roles.time_stamp)
dobito.set_role("sluzba", getml.data.roles.categorical)
dobito.set_role("kc_dobito", getml.data.roles.numerical)

dobito.set_unit("sluzba", "service")

dobito

In [ ]:
probehnuto.set_role("client_id", getml.data.roles.join_key)
probehnuto.set_role("month_year_datum_transakce", getml.data.roles.time_stamp)
probehnuto.set_role("sluzba", getml.data.roles.categorical)
probehnuto.set_role("kc_proklikano", getml.data.roles.target)

probehnuto.set_unit("sluzba", "service")

probehnuto

In [ ]:
probehnuto_mimo_penezenku.set_role("client_id", getml.data.roles.join_key)
probehnuto_mimo_penezenku.set_role("Month/Year", getml.data.roles.time_stamp)

probehnuto_mimo_penezenku

In [ ]:
split = getml.data.split.random(train=0.8, test=0.2)
split

## 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

### 2.1 Define relational model

In [ ]:
star_schema = getml.data.StarSchema(population=probehnuto, alias="population", split=split)

star_schema.join(
    probehnuto,
    on="client_id",
    time_stamps="month_year_datum_transakce",
    lagged_targets=True,
    horizon=getml.data.time.days(1),
)

star_schema.join(
    dobito,
    on="client_id",
    time_stamps="month_year_datum_transakce",
)

star_schema.join(
    probehnuto_mimo_penezenku,
    on="client_id", 
    time_stamps=("month_year_datum_transakce",  "Month/Year"),
)

star_schema

### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

We use the relboost algorithms for this problem. Because of the large number of keywords, we regularize the model a bit by requiring a minimum support for the keywords (`min_num_samples`).

In [ ]:
mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,    
    sampling_factor=0.1,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

__Build the pipeline__

In [ ]:
pipe1 = getml.Pipeline(
    tags=['fast_prop'],
    data_model=star_schema.data_model,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    predictors=[predictor],
    include_categorical=True,
)

pipe1

### 2.3 Model training

In [ ]:
pipe1.check(star_schema.train)

In [ ]:
pipe1.fit(star_schema.train)

### 2.4 Model evaluation

In [ ]:
pipe1.score(star_schema.test)

### 2.5 featuretools

In [ ]:
include = (getml.data.random() < 0.25)
include

In [ ]:
population_train_pd = star_schema.train.population[include].to_pandas()
population_test_pd = star_schema.test.population.to_pandas()

In [ ]:
population_train_pd["id"] = population_train_pd.index
population_test_pd["id"] = population_test_pd.index

In [ ]:
probehnuto_pd = probehnuto.drop(probehnuto.roles.unused).to_pandas()
dobito_pd = dobito.drop(dobito.roles.unused).to_pandas()
probehnuto_mimo_penezenku_pd = probehnuto_mimo_penezenku.drop(probehnuto_mimo_penezenku.roles.unused).to_pandas()

In [ ]:
def prepare_peripheral(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["id", "client_id", "month_year_datum_transakce"]],
        on="client_id"
    )

    peripheral_new = peripheral_new[
        peripheral_new["month_year_datum_transakce_x"] < peripheral_new["month_year_datum_transakce_y"]
    ]

    del peripheral_new["month_year_datum_transakce_y"]
    del peripheral_new["client_id"]

    return peripheral_new.rename({"month_year_datum_transakce_y": "month_year_datum_transakce"})

In [ ]:
def prepare_probehnuto_mimo_penezenku(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["id", "client_id", "month_year_datum_transakce"]],
        on="client_id"
    )

    peripheral_new = peripheral_new[
        peripheral_new["Month/Year"] < peripheral_new["month_year_datum_transakce"]
    ]

    del peripheral_new["month_year_datum_transakce"]
    del peripheral_new["client_id"]

    return peripheral_new

In [ ]:
dobito_train_pd = prepare_peripheral(dobito_pd, population_train_pd)
dobito_test_pd = prepare_peripheral(dobito_pd, population_test_pd)
dobito_train_pd

In [ ]:
probehnuto_train_pd = prepare_peripheral(probehnuto_pd, population_train_pd)
probehnuto_test_pd = prepare_peripheral(probehnuto_pd, population_test_pd)
probehnuto_train_pd

In [ ]:
probehnuto_mimo_penezenku_train_pd = prepare_probehnuto_mimo_penezenku(probehnuto_mimo_penezenku_pd, population_train_pd)
probehnuto_mimo_penezenku_test_pd = prepare_probehnuto_mimo_penezenku(probehnuto_mimo_penezenku_pd, population_test_pd)
probehnuto_mimo_penezenku_train_pd

In [ ]:
del population_train_pd["client_id"]
del population_test_pd["client_id"]

In [ ]:
population_train_pd

In [ ]:
dataframes_train = {
    "population" : (population_train_pd, "id"),
    "dobito": (dobito_train_pd, "index"),
    "probehnuto": (probehnuto_train_pd, "index"),
    "probehnuto_mimo_penezenku": (probehnuto_mimo_penezenku_train_pd, "index"),
}

In [ ]:
dataframes_test = {
    "population" : (population_test_pd, "id"),
    "dobito": (dobito_test_pd, "index"),
    "probehnuto": (probehnuto_test_pd, "index"),
    "probehnuto_mimo_penezenku": (probehnuto_mimo_penezenku_test_pd, "index"),
}

In [ ]:
relationships = [
    ("population", "id", "dobito", "id"),
    ("population", "id", "probehnuto", "id"),
    ("population", "id", "probehnuto_mimo_penezenku", "id"),
]

In [ ]:
featuretools_train_pd = featuretools.dfs(
    dataframes=dataframes_train,
    relationships=relationships,
    target_dataframe_name="population")[0]

In [ ]:
featuretools_test_pd = featuretools.dfs(
    dataframes=dataframes_test,
    relationships=relationships,
    target_dataframe_name="population")[0]

In [ ]:
featuretools_train = getml.data.DataFrame.from_pandas(featuretools_train_pd, "featuretools_train")
featuretools_test = getml.data.DataFrame.from_pandas(featuretools_test_pd, "featuretools_test")

In [ ]:
featuretools_train.set_role("kc_proklikano", getml.data.roles.target)
featuretools_train.set_role(featuretools_train.roles.unused_float, getml.data.roles.numerical)
featuretools_train.set_role(featuretools_train.roles.unused_string, getml.data.roles.categorical)

featuretools_train

In [ ]:
featuretools_test.set_role("kc_proklikano", getml.data.roles.target)
featuretools_test.set_role(featuretools_test.roles.unused_float, getml.data.roles.numerical)
featuretools_test.set_role(featuretools_test.roles.unused_string, getml.data.roles.categorical)

featuretools_test

We train an untuned XGBoostRegressor on top of featuretools' features, just like we have done for getML's features.

Since some of featuretools features are categorical, we allow the pipeline to include these features as well. Other features contain NaN values, which is why we also apply getML's Imputation preprocessor.

In [ ]:
data_model = getml.data.DataModel("population")

In [ ]:
imputation = getml.preprocessors.Imputation()

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe2 = getml.Pipeline(
    tags=['featuretools'],
    data_model=data_model,
    preprocessors=[imputation],
    predictors=[predictor],
    include_categorical=True,
)

pipe2

In [ ]:
pipe2.fit(featuretools_train)

In [ ]:
pipe2.score(featuretools_test)

### 2.6 Features

The most important feature looks as follows:

In [ ]:
pipe1.features.to_sql()[pipe1.features.sort(by="importances")[0].name]

### 2.7 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` and `spark` modules.

In [ ]:
# Creates a folder named seznam_pipeline containing
# the SQL code.
pipe1.features.to_sql().save("seznam_pipeline")

In [ ]:
pipe1.features.to_sql(dialect=getml.pipeline.dialect.spark_sql).save("seznam_spark")

### 2.8 Discussion

For a more convenient overview, we summarize our results into a table.

Name                 | R-squared  | RMSE   | MAE
-------------------- | ---------- | -------| ----
getML: FastProp      |     78.22% | 24,480 | 3,160
featuretools         |     63.24% | 31,655 | 5,167

## 3. Conclusion

We have benchmarked getML against featuretools on a dataset related to online transactions. We have found that getML outperforms featuretools by a wide margin.

## References

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).

# Next Steps

This tutorial benchmarked getML against academic state-of-the-art algorithms from relational learning literature and getML's qualities with respect to categorical data.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.